In [1]:
# Imports
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
import pandas as pd
import geopandas as gpd
import glob
import os

In [3]:
host = "localhost:5438"
database = "lair"
user = "postgres"
password = "postgres"

In [4]:
connection_string = f"postgresql://{user}:{password}@{host}/{database}"

In [5]:
engine = create_engine(connection_string)

In [6]:
base_path = r"D:\10_MILLENIUM\04_DADOS\01_DADOS_SANDRO\faltantes"
shapefiles = glob.glob(base_path + f'**/*.shp')
for shape in shapefiles:
    gdf = gpd.read_file(shape)
    gdf = gdf.to_crs(epsg=4326)
    gdf = gdf.explode(ignore_index=True)
    gdf.columns = [x.lower() for x in gdf.columns]
    gdf['geom'] = gdf['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
    gdf.drop(['geometry'], axis=1, inplace=True)
    gdf.to_sql('2013', engine, schema= 'qualidade_pastagem', if_exists='append', index=False, 
                         dtype={'geom': Geometry('POLYGON', srid= 4326)})